Import libraries

In [8]:

import pandas as pd 
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium


# definire il path in cui è stato clonato il repository
path =r"/mnt/c/Users/denise/Sviluppo/Repos/Address_verifier/"
import sys
sys.path.append(path)
import functions as mav 

In [9]:
# geolocator = Nominatim(user_agent="MyLocator")
# indirizzo = "Corso Italia, Genova"
# location = geolocator.geocode(indirizzo)

# if location:
#     print(f"indirizzo: {location.address}")
#     print(f"latitudine: {location.latitude}, longitudine: {location.longitude}")
# else:
#     print("Indirizzo non trovato")

In [10]:
list_indi = ['Viale Italia 17', 'Località Moggie, Zignago']


Function that checks for the existence of an address, takes in input a list of addresses

In [ ]:
output = mav.verifica_indirizzi(list_indi)

general function: 
it generates an html map where the addresses are marked based on coordinates obtained previously

In [12]:
map_interactive = mav.mappa_dinamica(output)

save map

In [ ]:
map_interactive.save(r"mappa_indirizzi.html")
print("La mappa è stata salvata come 'mappa_indirizzi.html'. Apri il file per visualizzarla.")

In [ ]:
import os
print(os.path.abspath("mappa_indirizzi.html"))